In [ ]:
# codes for auto-reload
# You don't need to re-import after modifying the modules
%load_ext autoreload
%autoreload 2

# 0. Make the relations based on Jac-Similarity

It takes too much time (about 1h) So, we recommend using the result.  
We enclose item_to_item_its_jac.csv and item_to_item_its_jac_with_user_item.csv in ./Dataset folder and recommend using it.

In [ ]:
import pandas as pd
from typing import Set
from collections import defaultdict
from tqdm import tqdm

path = "./Dataset"
user_item = pd.read_csv('./Dataset/user_item.csv', delimiter=',', names=['user_id', 'item_id'])
total_number_user: int = 53897
total_number_itemset: int = 27694
total_number_item: int = 42653

user_total_id: Set = set([i for i in range(total_number_user)])
user_item_user_id_set: Set = set(user_item['user_id'])
user_item_item_id_set: Set = set(user_item['item_id'])
nonmatching_user_count: Set = user_total_id - user_item_user_id_set

itemset_item_training = pd.read_csv('./Dataset/itemset_item_training.csv', delimiter=',', names=['itemset_id', 'item_id'])
itemset_item_id_set: Set = set(itemset_item_training['item_id'])
user_item_id_set: Set = set(user_item['item_id'])
total_item_id_set: Set = user_item_id_set.union(itemset_item_id_set)

row_connect_itemset_to_item_dataframe = itemset_item_training.groupby('itemset_id', as_index=False)['item_id'].agg(lambda x: list(sorted(x)))
row_connect_itemset_to_item_dataframe['item_count'] = row_connect_itemset_to_item_dataframe['item_id'].apply(lambda x: len(x))

row_connect_item_to_itemset_dataframe = itemset_item_training.groupby('item_id', as_index=False)['itemset_id'].agg(lambda x: list(sorted(x)))
row_connect_item_to_itemset_dataframe['itemset_count'] = row_connect_item_to_itemset_dataframe['itemset_id'].apply(lambda x: len(x))

row_connect_itemset_to_item_dataframe.insert(1, 'iset_id', row_connect_itemset_to_item_dataframe['itemset_id'])
row_connect_itemset_to_item_dataframe.set_index('iset_id', inplace=True)

for i in list(set(range(total_number_itemset)) - set(row_connect_itemset_to_item_dataframe['itemset_id'])):
    row_connect_itemset_to_item_dataframe.loc[i] = [i, [], 0]
row_connect_itemset_to_item_dataframe = row_connect_itemset_to_item_dataframe.sort_index()

ii_itemset_sim_check = list(row_connect_itemset_to_item_dataframe.itertuples(index=False))
ii_item_sim_check = list(row_connect_item_to_itemset_dataframe.itertuples(index=False))

ii_itemset_sim_check.sort(key=lambda x: x[0])
ii_item_sim_check.sort(key=lambda x: x[0])

jaccard_similarity = lambda s1, s2: len(s1 & s2) / len(s1 | s2)
overlap_similarity = lambda s1, s2: len(s1 & s2) / min(len(s1), len(s2))

connection_dict_jaccard_sim = defaultdict(float)

print("Make the relations based on itemset-item relation")
for itemset_data in tqdm(ii_itemset_sim_check):
    for i in range(len(itemset_data[1])):
        for j in range(i+1, len(itemset_data[1])):
            connection_dict_jaccard_sim[(ii_item_sim_check[itemset_data[1][i]][0], 
                                         ii_item_sim_check[itemset_data[1][j]][0])] = jaccard_similarity(set(ii_item_sim_check[itemset_data[1][i]][1]), 
                                                                                                      set(ii_item_sim_check[itemset_data[1][j]][1]))

print("Save the relations based on itemset-item relation")
with open(path + "/item_to_item_its_jac.csv", 'w') as f:
    for key, value in connection_dict_jaccard_sim.items():
        f.write(f'{int(key[0])},{int(key[1])},{value}\n')

jaccard_similarity = lambda s1, s2: len(s1 & s2) / len(s1 | s2)

# relation info
train_file = path + '/user_item.csv'
user_item = pd.read_csv(train_file, delimiter=',', names=['user_id', 'item_id'])

row_connect_user_to_item_dataframe = user_item.groupby('user_id', as_index=False)['item_id'].agg(lambda x: list(sorted(x)))
row_connect_user_to_item_dataframe['item_count'] = row_connect_user_to_item_dataframe['item_id'].apply(lambda x: len(x))

row_connect_item_to_user_dataframe = user_item.groupby('item_id', as_index=False)['user_id'].agg(lambda x: list(sorted(x)))
row_connect_item_to_user_dataframe['user_count'] = row_connect_item_to_user_dataframe['user_id'].apply(lambda x: len(x))

row_connect_user_to_item_dataframe.insert(1, 'u_id', row_connect_user_to_item_dataframe['user_id'])
row_connect_user_to_item_dataframe.set_index('u_id', inplace=True)

row_connect_item_to_user_dataframe.insert(1, 'i_id', row_connect_item_to_user_dataframe['item_id'])
row_connect_item_to_user_dataframe.set_index('i_id', inplace=True)

for i in list(set(range(total_number_user)) - set(row_connect_user_to_item_dataframe['user_id'])):
    row_connect_user_to_item_dataframe.loc[i] = [i, [], 0]
row_connect_user_to_item_dataframe = row_connect_user_to_item_dataframe.sort_index()

for i in list(set(range(total_number_item)) - set(row_connect_item_to_user_dataframe['item_id'])):
    row_connect_item_to_user_dataframe.loc[i] = [i, [], 0]
row_connect_item_to_user_dataframe = row_connect_item_to_user_dataframe.sort_index()

ui_user_sim_check = list(row_connect_user_to_item_dataframe.itertuples(index=False))
ui_item_sim_check = list(row_connect_item_to_user_dataframe.itertuples(index=False))

ui_user_sim_check.sort(key=lambda x: x[0])
ui_item_sim_check.sort(key=lambda x: x[0])
connection_dict_jaccard_sim = defaultdict(float)

item_value = defaultdict(float)
for item_data in tqdm(ui_user_sim_check):
    for i in range(len(item_data[1])):
        for j in range(i+1, len(item_data[1])):
            item_value[(ui_item_sim_check[item_data[1][i]][0], ui_item_sim_check[item_data[1][j]][0])] = jaccard_similarity(set(ui_item_sim_check[item_data[1][i]][1]), set(ui_item_sim_check[item_data[1][j]][1]))

with open(path + "/item_to_item_its_jac_with_user_item.csv", "w") as w:
    for (i, j), score in item_value.items():
        w.write("%d,%d,%f\n" %(i, j, score))

# 1. Maximum Spanning Tree (MST)  
  
We enclose mst.csv in ./Dataset folder and recommend using it.

In [ ]:
from Utils.mst import MST

mst = MST('./Dataset/item_to_item_its_jac_with_user_item.csv')
mst_make = mst.kruskal()
with open('./Dataset/mst.csv', 'w') as f:
    for i, j, _ in mst_make:
        f.write(f'{i},{j}\n')
        
extended_style = mst.extended_style_graph(mst_make)
with open('./Dataset/item_by_item_its_jac_with_mst.csv', 'w') as f:
    for (i, j), w in extended_style.items():
        f.write(f'{i},{j},{w}\n')

# 2. Hierarchical Community Detection  
  
The number of communities can vary depending on louvain, so the code was written based on the total number of clusters being 11776.  
We enclose louvain_community_feature.csv in ./Dataset folder and recommend using it.

In [ ]:
from Utils.item_community import community

com = community(weight=True)
louvain_community_list_each_step, hierarchical_community_data_each_item = com.make_hierarchical_community()

with open('./Dataset/louvain_community_feature.csv', 'w') as f:
    for i in hierarchical_community_data_each_item:
        f.write(f"{','.join([str(i)]+[str(item) for item in hierarchical_community_data_each_item[i]])}\n")

# 3. Get Item Embedding of CASG GNN  
  
We enclose CASG_GNN_ITEM_OUT.npy in ./Dataset folder and recommend using it.

In [ ]:
import torch
import numpy as np
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
# from pytorch_lightning.metrics.functional import accuracy
from tqdm import tqdm
from Datasets import Task2Dataset
from Models.Task2 import Task2Net

if torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'

###########################################################
batch_size = 65536
lr = 1e-3
epoch = 3000
scheduler_step = 300
scheduler_decay = 0.9

resample_rate = 0.35 # dataset에서 label로 쓰일 비율
resample_epoch = 10

save_folder = "./GCN_3L_CASG"

###########################################################

dataset = Task2Dataset("Dataset/")
dataset.resample(resample_rate)
g = dataset.g.to(device)
train_g = dataset.train_g.to(device)

train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
model = Task2Net(dataset.g.to(device)).to(device)

# create optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_decay)

# Our model
checkpoint = torch.load('CASG_model_0.1437.tar')
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])
print(model)

ce_loss = nn.CrossEntropyLoss()

for itemset_id, query, pos, neg, query_items, lengths in train_dataloader:
    pass

model.eval()
with torch.no_grad():
    labels = []
    batched_query = []
    query_items = []
    lengths = []
    for itemset_id, querys in dataset.valid_itemset_items.items():
        query = np.zeros(dataset.n_items, dtype=np.float32)
        label = dataset.valid_itemset_label[itemset_id]
        query[querys] = 1

        batched_query.append(query)
        labels.append(label)
        query_items.append(querys + [0 for _ in range(5 - len(querys))])
        lengths.append(len(querys))

    # import pdb; pdb.set_trace()
    query_embeds, pos_embeds, _, logit = model(g, torch.tensor(batched_query).to(device), 
                                               torch.tensor(list(range(dataset.n_items))).to(device), 
                                               neg.to(device), 
                                               torch.tensor(query_items).to(device), 
                                               torch.tensor(lengths).to(device))
    np.save("./Dataset/CASG_GNN_ITEM_OUT.npy", pos_embeds.detach().cpu().numpy())